In [1]:
import pandas as pd
import requests

local imports

In [2]:
from main import make_query, parse_OTP_response
from utils import plot
from utils import get_config

set up

In [3]:
PATH = 'georequests_O(area10)_D.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

params = get_config('default_SUM.json')
# params = make_paths(params) # update file paths
# VoT - value of time, param

In [4]:
df = pd.read_csv(PATH)
df.head(3)

,Unnamed: 0,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
0,0,19.88948,50.01844,19.954225,50.045635,2024-03-28 07:54:33,True,3.0,1560.0,274.988511,0.0,1320.0,2.0,"[['TRAM', 1320, 6352], ['WALK', 238, 274]]"
1,1,19.88948,50.01844,19.934195,50.070585,2024-03-28 07:52:19,True,3.0,2149.0,406.611526,0.0,1800.0,2.0,"[['TRAM', 1800, 9003], ['WALK', 347, 406]]"
2,2,19.88948,50.01844,19.896833,50.023390,2024-03-28 08:05:43,True,3.0,478.0,439.951789,0.0,120.0,2.0,"[['TRAM', 120, 1182], ['WALK', 356, 439]]"


In [5]:
df1 = df.filter(['x','y','desti_x', 'desti_y', 'treq'], axis = 1).rename(
    columns = {'x': 'origin_x', 'y': 'origin_y', 'desti_x': 'destination_x', 'desti_y': 'destination_y'})

df1['treq'] = pd.to_datetime(df1.treq)
df1.head(3)

,Unnamed: 0,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
0,0,19.88948,50.01844,19.954225,50.045635,2024-03-28 07:54:33,True,3.0,1560.0,274.988511,0.0,1320.0,2.0,"[['TRAM', 1320, 6352], ['WALK', 238, 274]]"
1,1,19.88948,50.01844,19.934195,50.070585,2024-03-28 07:52:19,True,3.0,2149.0,406.611526,0.0,1800.0,2.0,"[['TRAM', 1800, 9003], ['WALK', 347, 406]]"
2,2,19.88948,50.01844,19.896833,50.023390,2024-03-28 08:05:43,True,3.0,478.0,439.951789,0.0,120.0,2.0,"[['TRAM', 120, 1182], ['WALK', 356, 439]]"


<h3>query OTP for each request</h3>

In [6]:
df2 = df1.apply(lambda row: 
                 parse_OTP_response(requests.get(OTP_API, params=make_query(row.squeeze())).json()), axis=1)


In [7]:
# test
df2[0]

{'success': True,
 'n_itineraries': 3,
 'duration': 1560,
 'walkDistance': 274.98851127159645,
 'transfers': 0,
 'transitTime': 1320,
 'waitingTime': 2,
 'modes': [['TRAM', 1320, 6352], ['WALK', 238, 274]]}

In [8]:
for name in list(df2[0].keys()):
    vals = []
    for i in range(len(df2)):
        if df2[i]['success'] == True:
            vals.append(df2[i][name])
        else:
            vals.append(None)
    df1[name] = vals
df1[:3]

,Unnamed: 0,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
0,0,19.88948,50.01844,19.954225,50.045635,2024-03-28 07:54:33,True,3.0,1560.0,274.988511,0.0,1320.0,2.0,"[[TRAM, 1320, 6352], [WALK, 238, 274]]"
1,1,19.88948,50.01844,19.934195,50.070585,2024-03-28 07:52:19,True,3.0,2149.0,406.611526,0.0,1800.0,2.0,"[[TRAM, 1800, 9003], [WALK, 347, 406]]"
2,2,19.88948,50.01844,19.896833,50.023390,2024-03-28 08:05:43,True,3.0,478.0,439.951789,0.0,120.0,2.0,"[[TRAM, 120, 1182], [WALK, 356, 439]]"


In [9]:
df1[61 : 64]

,Unnamed: 0,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
61,61,19.88948,50.01844,19.896833,50.023390,2024-03-28 07:48:47,True,3.0,478.0,439.951789,0.0,120.0,2.0,"[[TRAM, 120, 1182], [WALK, 356, 439]]"
62,62,19.88948,50.01844,20.077178,50.072775,2024-03-28 08:02:50,None,NaN,NaN,NaN,NaN,NaN,NaN,None
63,63,19.88948,50.01844,19.896833,50.023390,2024-03-28 07:54:18,True,3.0,478.0,439.951789,0.0,120.0,2.0,"[[TRAM, 120, 1182], [WALK, 356, 439]]"


In [10]:
# test
row = df1.sample(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route


{'success': True,
 'n_itineraries': 3,
 'duration': 952,
 'walkDistance': 850.6867368309817,
 'transfers': 0,
 'transitTime': 240,
 'waitingTime': 2,
 'modes': [['TRAM', 240, 2187], ['WALK', 710, 850]]}

In [11]:
# visualization
plot(r.json()['plan'], color = 'green')

Trip from (19.8895,50.0184) to (19.9146,50.0242) at 1711608480000. 
3 connections found. 
Best one is 16min (851m walk, 0 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	TRAM	2188	240
2	WALK	850	710


In [12]:
def PT_utility(requests, params):
    if 'walkDistance' in requests.columns:
        requests = requests
        requests['PT_fare'] = 1 + requests.transitTime * params.avg_speed/1000 * params.ticket_price
        requests['u_PT'] = requests['PT_fare'] + \
                           params.VoT * (params.walk_factor * requests.walkDistance / params.speeds.walk +
                                           params.wait_factor * requests.waitingTime +
                                           params.transfer_penalty * requests.transfers + requests.transitTime)
    return requests

In [13]:
reqs = PT_utility(df1, params)

In [14]:

reqs

,Unnamed: 0,origin_x,origin_y,destination_x,destination_y,treq,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes,PT_fare,u_PT
0,0,19.88948,50.01844,19.954225,50.045635,2024-03-28 07:54:33,True,3.0,1560.0,274.988511,0.0,1320.0,2.0,"[[TRAM, 1320, 6352], [WALK, 238, 274]]",2.386,8.624100
1,1,19.88948,50.01844,19.934195,50.070585,2024-03-28 07:52:19,True,3.0,2149.0,406.611526,0.0,1800.0,2.0,"[[TRAM, 1800, 9003], [WALK, 347, 406]]",2.890,11.575901
2,2,19.88948,50.01844,19.896833,50.023390,2024-03-28 08:05:43,True,3.0,478.0,439.951789,0.0,120.0,2.0,"[[TRAM, 120, 1182], [WALK, 356, 439]]",1.126,4.126385
3,3,19.88948,50.01844,19.895333,50.085894,2024-03-28 07:49:17,True,3.0,2685.0,398.556195,2.0,2100.0,239.0,"[[TRAM, 780, 5014], [WALK, 3, 0], [BUS, 660, 2...",3.205,18.052911
4,4,19.88948,50.01844,19.932209,50.056612,2024-03-28 08:03:51,True,3.0,1825.0,391.812782,0.0,1500.0,2.0,"[[TRAM, 1500, 7605], [WALK, 323, 391]]",2.575,10.124575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,161,19.88948,50.01844,19.951528,50.063938,2024-03-28 07:59:35,True,3.0,1961.0,481.113308,0.0,1560.0,2.0,"[[TRAM, 1560, 8231], [WALK, 399, 480]]",2.638,10.918494
162,162,19.88948,50.01844,19.861527,50.017915,2024-03-28 07:58:17,True,3.0,1193.0,1205.108714,0.0,240.0,2.0,"[[WALK, 50, 43], [BUS, 240, 1511], [WALK, 901,...",1.252,9.135801
163,163,19.88948,50.01844,19.896833,50.023390,2024-03-28 07:48:07,True,3.0,478.0,439.951789,0.0,120.0,2.0,"[[TRAM, 120, 1182], [WALK, 356, 439]]",1.126,4.126385
164,164,19.88948,50.01844,20.016993,50.086808,2024-03-28 08:01:18,True,3.0,3236.0,474.108376,0.0,2820.0,2.0,"[[TRAM, 2820, 14808], [WALK, 414, 473]]",3.961,16.610632


In [15]:
reqs.to_csv("PT_utility_HD(area10).csv")